In [1]:
import torch
import flwr as fl
from torch.utils.data import Dataset, Subset
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor
from client import get_client_generator, weighted_average_accuracy
from dataset import partition_dataset
from util import seed_everything
from flwr.server.strategy import FedAvg

transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
train_dataset = MNIST("./mnist", train=True, download=True, transform=transform)
val_dataset = MNIST("./mnist", train=False, transform=transform)

def complete_run(seed):
    seed_everything(seed)

    num_clients = 10
    train_datasets = partition_dataset(train_dataset, num_clients)
    val_datasets = partition_dataset(val_dataset, num_clients)
    train_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) for dataset in train_datasets]
    val_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16) for dataset in val_datasets]
    client_resources = None
    client_fn = get_client_generator(train_dataloaders, val_dataloaders)
    client_config = {
        "lr": 0.05,
        "epochs": 1,
    }
    strategy = FedAvg(
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        fraction_fit=0.5,
        fraction_evaluate=0.5,
        on_fit_config_fn=lambda _: client_config,
        on_evaluate_config_fn=lambda _: client_config,
        evaluate_metrics_aggregation_fn=weighted_average_accuracy,
    )
    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=10),
        client_resources=client_resources,
        strategy=strategy
    )
    return hist

In [2]:
run1 = complete_run(0)

INFO flwr 2023-03-04 20:31:09,305 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-03-04 20:31:11,339	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-04 20:31:12,799 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'CPU': 10.0, 'object_store_memory': 2147483648.0, 'memory': 18234695680.0}
INFO flwr 2023-03-04 20:31:12,799 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-04 20:31:12,800 | server.py:270 | Requesting initial parameters from one random client
INFO flwr 2023-03-04 20:31:13,418 | server.py:274 | Received initial parameters from one random client
INFO flwr 2023-03-04 20:31:13,418 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-04 20:31:13,418 | server.py:101 | FL starting
DEBUG flwr 2023-03-04 20:31:13,419 | server.py:215 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-03-04 20:31:22,250 | server.py:229 | fit_r

In [3]:
run2 = complete_run(0)

INFO flwr 2023-03-04 20:32:46,528 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-03-04 20:32:51,661	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-03-04 20:32:53,048 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2147483648.0, 'CPU': 10.0, 'memory': 18565985076.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-03-04 20:32:53,049 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-04 20:32:53,049 | server.py:270 | Requesting initial parameters from one random client
INFO flwr 2023-03-04 20:32:53,835 | server.py:274 | Received initial parameters from one random client
INFO flwr 2023-03-04 20:32:53,835 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-04 20:32:53,835 | server.py:101 | FL starting
DEBUG flwr 2023-03-04 20:32:53,836 | server.py:215 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-03-04 20:33:02,790 | server.py:229 | fit_r